# Trim and resurface designs

### Boilerplate

In [ ]:
%load_ext lab_black
# python internal
import collections
import copy
import gc
from glob import glob
import h5py
import itertools
import os
import random
import re
import socket
import shutil
import subprocess
import sys

# conda/pip
import dask
import graphviz
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy
import seaborn as sns
from tqdm import tqdm

# special packages on the DIGS
import py3Dmol
import pymol
import pyrosetta

# notebook magic
%matplotlib inline
%load_ext autoreload
%autoreload 2

print(os.getcwd())
print(socket.gethostname())

### Make function to trim and resurface
We want to trim designs that are close to 162 aa to 162. May also want to pad designs with GS at a later step.
SAP constraint will prevent grease balls, should probably use fragments too to help folding

In [2]:
from pyrosetta.distributed.packed_pose.core import PackedPose
from pyrosetta.distributed import requires_init


@requires_init
def detail_design(packed_pose_in: PackedPose, **kwargs) -> PackedPose:
    """
    Truncate designs > 162aa & < 176aa to 162aa.
    Resurface with SAP constraint.
    Recompute score metrics.
    """
    import bz2
    import pyrosetta
    import pyrosetta.distributed.io as io
    from pyrosetta.distributed.tasks.rosetta_scripts import (
        SingleoutputRosettaScriptsTask,
    )

    def trim(seq_length: int) -> list:
        target = 162
        maximum = 175
        if seq_length > maximum or seq_length < target:
            to_trim = [str(seq_length)]
            return to_trim
        else:
            amount_to_trim = seq_length - target
            n_term_trim = int(amount_to_trim / 2)
            c_term_trim = amount_to_trim - n_term_trim
            all_resis = list(range(1, seq_length + 1))
            to_trim = [str(x) for x in all_resis[:n_term_trim]] + [
                str(x) for x in all_resis[-c_term_trim:]
            ]
            return to_trim

    if packed_pose_in == None:
        file = kwargs["-s"]
        with open(file, "rb") as f:
            packed_pose_in = io.pose_from_pdbstring(bz2.decompress(f.read()).decode())
        scores = dict(pyrosetta.distributed.cluster.get_scores_dict(file)["scores"])
    else:
        raise RuntimeError("Need to supply an input")

    to_trim = trim(int(float(scores["chB_start"]) - 1))
    offset = len(to_trim)
    new_interface_hydrophobics = ",".join(
        [str(int(x) - offset) for x in scores["interface_hydrophobics"].split(",")]
    )
    scores["interface_hydrophobics"] = new_interface_hydrophobics
    xml = """
    <ROSETTASCRIPTS>
        <SCOREFXNS>
            <ScoreFunction name="sfxn" weights="beta_nov16" />
            <ScoreFunction name="sfxn_soft" weights="beta_nov16_soft" />
            <ScoreFunction name="sfxn_fa_atr" weights="empty" >
                <Reweight scoretype="fa_atr" weight="1" />
            </ScoreFunction>
            <ScoreFunction name="sfxn_relax" weights="beta_nov16" >
                <Reweight scoretype="arg_cation_pi" weight="3" />
                <Reweight scoretype="approximate_buried_unsat_penalty" weight="5" />
                <Set approximate_buried_unsat_penalty_burial_atomic_depth="3.5" />
                <Set approximate_buried_unsat_penalty_hbond_energy_threshold="-0.5" />
            </ScoreFunction>
            <ScoreFunction name="sfxn_design" weights="beta_nov16" >
                <Reweight scoretype="res_type_constraint" weight="1.5" />
                <Reweight scoretype="aa_composition" weight="1.0" />
                <Reweight scoretype="sap_constraint" weight="1.0" />
                <Reweight scoretype="netcharge" weight="1.0" />
                <Reweight scoretype="arg_cation_pi" weight="3" />
                <Reweight scoretype="approximate_buried_unsat_penalty" weight="5" />
                <Set approximate_buried_unsat_penalty_burial_atomic_depth="3.5" />
                <Set approximate_buried_unsat_penalty_hbond_energy_threshold="-0.5" />
                <Set approximate_buried_unsat_penalty_hbond_bonus_cross_chain="-1" />
            </ScoreFunction>
            <ScoreFunction name="vdw_sol" weights="empty" >
                <Reweight scoretype="fa_atr" weight="1.0" />
                <Reweight scoretype="fa_rep" weight="0.55" />
                <Reweight scoretype="fa_sol" weight="1.0" />
            </ScoreFunction>
        </SCOREFXNS>
        <TASKOPERATIONS>
            <SelectBySASA name="PR_monomer_core_sel" mode="sc" state="monomer" probe_radius="2.2" core_asa="15" surface_asa="15" core="0" boundary="1" surface="1" verbose="0" />
        </TASKOPERATIONS>
        <RESIDUE_SELECTORS>
            <Chain name="chainA" chains="A"/>
            <Chain name="chainB" chains="B"/>
            <Neighborhood name="interface_chA" selector="chainB" distance="10.0" />
            <Neighborhood name="interface_chB" selector="chainA" distance="10.0" />
            <And name="AB_interface" selectors="interface_chA,interface_chB" />
            <Not name="Not_interface" selector="AB_interface" />
            <And name="actual_interface_chA" selectors="AB_interface,chainA" />
            <And name="actual_interface_chB" selectors="AB_interface,chainB" />
            <And name="chainB_not_interface" selectors="Not_interface,chainB" />
            <Not name="not_chB" selector="chainB" />
            <ResidueName name="pro_and_gly_positions" residue_name3="PRO,GLY,CYS" />
            <ResidueName name="apolar" residue_name3="ALA,CYS,PHE,ILE,LEU,MET,THR,PRO,VAL,TRP,TYR" />
            <Not name="polar" selector="apolar" />
            <InterfaceByVector name="interface_by_vector" cb_dist_cut="11" nearby_atom_cut="5.5" vector_angle_cut="75" vector_dist_cut="9" grp1_selector="actual_interface_chA" grp2_selector="actual_interface_chB"/>
            <Task name="all_cores" fixed="true" task_operations="PR_monomer_core_sel" packable="false" designable="false"/>
            <And name="for_hydrophobic" selectors="actual_interface_chA,interface_by_vector">
                <Not selector="all_cores" />
            </And>            
            <Index name="patchdock_res" resnums="{interface_hydrophobics}" />
            <Index name="pocket" resnums="{interface_hydrophobics}" />
            <And name="target_not_pocket" selectors="chainB">
                <Not selector="pocket" />
            </And>
            <Layer name="surface" select_core="false" select_boundary="false" select_surface="true" use_sidechain_neighbors="true"/>
            <Layer name="boundary" select_core="false" select_boundary="true" select_surface="false" use_sidechain_neighbors="true"/>
            <Layer name="core" select_core="true" select_boundary="false" select_surface="false" use_sidechain_neighbors="true"/>
            <SecondaryStructure name="sheet" overlap="0" minH="3" minE="2" include_terminal_loops="false" use_dssp="true" ss="E"/>
            <SecondaryStructure name="entire_loop" overlap="0" minH="3" minE="2" include_terminal_loops="true" use_dssp="true" ss="L"/>
            <SecondaryStructure name="entire_helix" overlap="0" minH="3" minE="2" include_terminal_loops="false" use_dssp="true" ss="H"/>
            <And name="helix_cap" selectors="entire_loop">
                <PrimarySequenceNeighborhood lower="1" upper="0" selector="entire_helix"/>
            </And>
            <And name="helix_start" selectors="entire_helix">
                <PrimarySequenceNeighborhood lower="0" upper="1" selector="helix_cap"/>
            </And>
            <And name="helix" selectors="entire_helix">
                <Not selector="helix_start"/>
            </And>
            <And name="loop" selectors="entire_loop">
                <Not selector="helix_cap"/>
            </And>
            <And name="surface_not_int" selectors="surface,chainA">
                <Not selector="AB_interface"/>
            </And>
            <And name="boundary_not_int" selectors="boundary,chainA">
                <Not selector="AB_interface"/>
            </And>
            <Or name="resurfaceable" selectors="surface_not_int,boundary_not_int" />
            <Not name="not_resurfaceable" selector="resurfaceable" />
            <Index name="to_trim" resnums="{indices_to_delete}" />
        </RESIDUE_SELECTORS>
        <TASKOPERATIONS>
            <DesignRestrictions name="layer_design_F_boundary_M">
                <Action selector_logic="not_chB AND surface AND helix_start"  aas="DEHKPQR"/>
                <Action selector_logic="not_chB AND surface AND helix"        aas="EHKQR"/>
                <Action selector_logic="not_chB AND surface AND sheet"        aas="EHKNQRST"/>
                <Action selector_logic="not_chB AND surface AND loop"         aas="DEGHKNPQRST"/>
                <Action selector_logic="not_chB AND boundary AND helix_start" aas="ADEFHIKLMNPQRSTVWY"/>
                <Action selector_logic="not_chB AND boundary AND helix"       aas="ADEFHIKLMNQRSTVWY"/>
                <Action selector_logic="not_chB AND boundary AND sheet"       aas="DEFHIKLMNQRSTVWY"/>
                <Action selector_logic="not_chB AND boundary AND loop"        aas="ADEFGHIKLNPQRSTVWY"/>
                <Action selector_logic="not_chB AND core AND helix_start"     aas="AFILMPVWY"/>
                <Action selector_logic="not_chB AND core AND helix"           aas="AFILMVWYDENQSTH"/>
                <Action selector_logic="not_chB AND core AND sheet"           aas="FILMVWYDENQSTH"/>
                <Action selector_logic="not_chB AND core AND loop"            aas="AFGILPVWYDENQSTH"/>
                <Action selector_logic="not_chB AND helix_cap"                aas="DNST"/>
            </DesignRestrictions>
        </TASKOPERATIONS>
        <TASKOPERATIONS>
            <PruneBuriedUnsats name="prune_buried_unsats" allow_even_trades="false" atomic_depth_cutoff="3.5" minimum_hbond_energy="-0.5" />
            <IncludeCurrent name="current" />
            <LimitAromaChi2 name="limitchi2" chi2max="110" chi2min="70" include_trp="True" />
            <ExtraRotamersGeneric name="ex1_ex2" ex1="1" ex2aro="1" />
            <OperateOnResidueSubset name="restrict_target_not_interface" selector="chainB_not_interface">
                <PreventRepackingRLT/>
            </OperateOnResidueSubset>
            <OperateOnResidueSubset name="restrict_interface" selector="AB_interface">
                <RestrictToRepackingRLT/>
            </OperateOnResidueSubset>
            <OperateOnResidueSubset name="restrict_target2repacking" selector="chainB">
                <RestrictToRepackingRLT/>
            </OperateOnResidueSubset>
            <OperateOnResidueSubset name="resurface_only" selector="not_resurfaceable">
                <RestrictToRepackingRLT/>
            </OperateOnResidueSubset>
            <OperateOnResidueSubset name="lock_core" selector="core">
                <PreventRepackingRLT/>
            </OperateOnResidueSubset>
            <DisallowIfNonnative name="disallow_GLY" resnum="0" disallow_aas="G" />
            <DisallowIfNonnative name="disallow_PRO" resnum="0" disallow_aas="P" />
            <OperateOnResidueSubset name="restrict_PRO_GLY" selector="pro_and_gly_positions">
                <PreventRepackingRLT/>
            </OperateOnResidueSubset>
            <SelectBySASA name="PR_monomer_core" mode="sc" state="monomer" probe_radius="2.2" core_asa="10" surface_asa="10" core="0" boundary="1" surface="1" verbose="0" />
            <ProteinInterfaceDesign name="pack_long" design_chain1="0" design_chain2="0" jump="1" interface_distance_cutoff="15"/>
        </TASKOPERATIONS>
        <MOVERS>
            <DeleteRegionMover name="delete_target_not_pocket" residue_selector="target_not_pocket" rechain="false" />
            <SwitchChainOrder name="chain1onlypre" chain_order="1" />
            <ScoreMover name="scorepose" scorefxn="sfxn" verbose="false" />
            <ParsedProtocol name="chain1only">
                <Add mover="chain1onlypre" />
                <Add mover="scorepose" />
            </ParsedProtocol>
            <TaskAwareMinMover name="min" scorefxn="sfxn" bb="0" chi="1" task_operations="pack_long" />
            <DeleteRegionMover name="delete_polar" residue_selector="polar" rechain="false" />
            <AddSapConstraintMover name="add_sap" speed="lightning" sap_goal="25" penalty_per_sap="1" />
            <AddNetChargeConstraintMover name="charged" filename="/mnt/home/bcov/sc/scaffold_comparison/data/net_charges/minus7.charge" selector="chainA" />
            <StructProfileMover name="genProfile" add_csts_to_pose="1" consider_topN_frags="100" eliminate_background="0" ignore_terminal_residue="1" only_loops="0" burialWt="0" RMSthreshold="0.6" residue_selector="chainA" />
            <ClearConstraintsMover name="clear_constraints" />
            <SavePoseMover name="save" restore_pose="0" reference_name="before_relax" />
        </MOVERS>
        <FILTERS>
            <Rmsd name="rmsd" reference_name="before_relax" confidence="0" />
            <Sasa name="interface_buried_sasa" confidence="0" />
            <Ddg name="ddg"  threshold="-10" jump="1" repeats="5" repack="1" relax_mover="min" confidence="0" scorefxn="sfxn" extreme_value_removal="1" />
            <ShapeComplementarity name="interface_sc" verbose="0" min_sc="0.55" write_int_area="1" write_median_dist="1" jump="1" confidence="0"/>
            <ShapeComplementarity name="interface_sc_for_fr1" verbose="0" min_sc="0.55" write_int_area="1" write_median_dist="1" jump="1" confidence="0"/>
            <ScoreType name="total_score_MBF" scorefxn="sfxn" score_type="total_score" threshold="0" confidence="0" />
            <MoveBeforeFilter name="total_score_monomer" mover="chain1only" filter="total_score_MBF" confidence="0" />
            <ResidueCount name="res_count_MBF" max_residue_count="9999" confidence="0"/>
            <MoveBeforeFilter name="res_count_monomer" mover="chain1only" filter="res_count_MBF" confidence="0" />
            <CalculatorFilter name="score_per_res" equation="total_score_monomer / res" threshold="-3.5" confidence="0">
                <Var name="total_score_monomer" filter="total_score_monomer"/>
                <Var name="res" filter="res_count_monomer"/>
            </CalculatorFilter>
            <BuriedUnsatHbonds name="buns_heavy_ball_1.1D" use_reporter_behavior="true" report_all_heavy_atom_unsats="true" scorefxn="sfxn" residue_selector="AB_interface" ignore_surface_res="false" print_out_info_to_pdb="true" confidence="0" use_ddG_style="true" burial_cutoff="0.01" dalphaball_sasa="true" probe_radius="1.1" max_hbond_energy="1.5" burial_cutoff_apo="0.2" />
            <BuriedUnsatHbonds name="sbuns5.0_heavy_ball_1.1D" use_reporter_behavior="true" report_all_heavy_atom_unsats="true" scorefxn="sfxn" residue_selector="AB_interface" ignore_surface_res="false" print_out_info_to_pdb="true" confidence="0" use_ddG_style="true" burial_cutoff="0.01" dalphaball_sasa="true" probe_radius="1.1" atomic_depth_selection="5.0" atomic_depth_deeper_than="false" burial_cutoff_apo="0.2" atomic_depth_resolution="0.49" max_hbond_energy="1.5"/>
            <BuriedUnsatHbonds name="vbuns5.0_heavy_ball_1.1D" use_reporter_behavior="true" report_all_heavy_atom_unsats="true" scorefxn="sfxn" residue_selector="AB_interface" ignore_surface_res="false" print_out_info_to_pdb="true" confidence="0" use_ddG_style="true" dalphaball_sasa="true" probe_radius="1.1" atomic_depth_selection="5.0" burial_cutoff="1000" burial_cutoff_apo="0.2" atomic_depth_apo_surface="5.5" atomic_depth_resolution="0.49" max_hbond_energy="1.5"/>
            <InterfaceHydrophobicResidueContacts name="hydrophobic_residue_contacts" target_selector="chainB" binder_selector="chainA" scorefxn="sfxn_soft" confidence="0"/> 
            <ContactMolecularSurface name="contact_molecular_surface" distance_weight="0.5" target_selector="chainA" binder_selector="chainB" confidence="0" />
            <ContactMolecularSurface name="contact_patch" distance_weight="0.5" target_selector="patchdock_res" binder_selector="chainA" confidence="0" />
            <Ddg name="ddg_hydrophobic_pre"  threshold="-10" jump="1" repeats="1" repack="0" confidence="0" scorefxn="vdw_sol" />
            <MoveBeforeFilter name="ddg_hydrophobic" mover="delete_polar" filter="ddg_hydrophobic_pre" confidence="0"/>
            <Ddg name="fa_atr_pocket_pre"  threshold="-15" jump="1" repeats="1" repack="0" confidence="0" scorefxn="sfxn_fa_atr" />
            <MoveBeforeFilter name="fa_atr_pocket" mover="delete_target_not_pocket" filter="fa_atr_pocket_pre" confidence="0"/>
            <SSShapeComplementarity name="ss_sc_pre" verbose="0" confidence="0" />
            <MoveBeforeFilter name="ss_sc" mover="chain1only" filter="ss_sc_pre" confidence="0"/>
            <Time name="timed"/>
        </FILTERS>
        <MOVERS>
            <DeleteRegionMover name="trim" residue_selector="to_trim" rechain="false" />
            <SwitchChainOrder name="rechain" chain_order="12"/>
            <FastDesign name="resurface" scorefxn="sfxn_design" repeats="2" task_operations="current,limitchi2,ex1_ex2,restrict_interface,restrict_target2repacking,resurface_only,lock_core,disallow_GLY,disallow_PRO,PR_monomer_core,restrict_PRO_GLY,prune_buried_unsats,layer_design_F_boundary_M" batch="false" ramp_down_constraints="false" cartesian="false" bondangle="false" bondlength="false" min_type="dfpmin_armijo_nonmonotone" relaxscript="InterfaceDesign2019" >
                <MoveMap name="MM" >
                        <Chain number="1" chi="true" bb="true" />
                        <Chain number="2" chi="true" bb="false" />
                        <Jump number="1" setting="true" />
                </MoveMap>
            </FastDesign>
            <FastRelax name="relax" scorefxn="sfxn_relax" repeats="1" batch="false" ramp_down_constraints="false" cartesian="false" bondangle="false" bondlength="false" min_type="dfpmin_armijo_nonmonotone" task_operations="current,ex1_ex2,restrict_target_not_interface,limitchi2,prune_buried_unsats" >
                <MoveMap name="MM" >
                    <Chain number="1" chi="true" bb="true" />
                    <Chain number="2" chi="true" bb="true" />
                    <Jump number="1" setting="true" />
                </MoveMap>
            </FastRelax>
        </MOVERS>
        <APPLY_TO_POSE>
        </APPLY_TO_POSE>
        <PROTOCOLS>
            <Add filter="timed" />
            <Add mover="trim" />
            <Add mover="rechain" />
            Add mover="genProfile" />
            <Add mover="add_sap" />
            <Add mover="charged" />
            <Add mover="resurface" />
            <Add mover="clear_constraints" />
            <Add mover="save" />
            <Add mover="relax" />
            <Add filter_name="rmsd" />
            <Add filter_name="interface_buried_sasa" />   
            <Add filter_name="ddg" />                     
            <Add filter="ddg_hydrophobic" />              
            <Add filter_name="interface_sc" />            
            <Add filter_name="score_per_res" />          
            <Add filter="vbuns5.0_heavy_ball_1.1D" />    
            <Add filter="sbuns5.0_heavy_ball_1.1D" />    
            <Add filter="buns_heavy_ball_1.1D" />    
            <Add filter="hydrophobic_residue_contacts" />
            <Add filter="fa_atr_pocket" />               
            <Add filter="contact_molecular_surface" />
            <Add filter="contact_patch" />   
            <Add filter="ss_sc" />   
            <Add filter="timed" />
        </PROTOCOLS>
        <OUTPUT />
    </ROSETTASCRIPTS>
    """.format(
        indices_to_delete=",".join(to_trim),
        interface_hydrophobics=scores["interface_hydrophobics"],
    )
    detailer = SingleoutputRosettaScriptsTask(xml)
    designed_ppose = detailer(packed_pose_in.pose.clone())
    new_scores = dict(designed_ppose.pose.scores)
    scores.update(new_scores)
    scores["interface_hydrophobics"] = new_interface_hydrophobics
    score_xml = """
    <ROSETTASCRIPTS>
        <SCOREFXNS>
            <ScoreFunction name="sfxn" weights="beta_nov16" />
        </SCOREFXNS>
        <MOVERS>
            <SwitchChainOrder name="chain1only" chain_order="1" />
            <RollMover name="move_chainA_far_away" chain="1" min_angle="0" max_angle="0" axis="x" >
                <translate x="1000" y="1000" z="1000" /> 
            </RollMover>
        </MOVERS>
        <RESIDUE_SELECTORS>
            <Chain name="chainA" chains="A"/>
            <Chain name="chainB" chains="B"/>
            <Neighborhood name="interface_chA" selector="chainB" distance="10.0" />
            <Neighborhood name="interface_chB" selector="chainA" distance="10.0" />
            <And name="AB_interface" selectors="interface_chA,interface_chB" />
            <Not name="Not_interface" selector="AB_interface" />
            <And name="actual_interface_chA" selectors="AB_interface,chainA" />
            <And name="actual_interface_chB" selectors="AB_interface,chainB" />
            <ResidueName name="apolar" residue_name3="ALA,CYS,PHE,ILE,LEU,MET,THR,PRO,VAL,TRP,TYR" />
            <And name="apolar_A" selectors="apolar,actual_interface_chA" />
            <And name="apolar_B" selectors="apolar,actual_interface_chB" />
            <Index name="patchdock_res" resnums="{interface_hydrophobics}" />
            <True name="true_sel" />
        </RESIDUE_SELECTORS>
        <SIMPLE_METRICS>
            <SapScoreMetric name="sap_score" score_selector="chainA" />
            <SapScoreMetric name="sap_score_target" score_selector="chainB" />
            <SapScoreMetric name="binder_blocked_sap" score_selector="chainA" sap_calculate_selector="chainA" sasa_selector="true_sel" />
            <SapScoreMetric name="target_blocked_sap" score_selector="chainB" sap_calculate_selector="chainB" sasa_selector="true_sel" />
        </SIMPLE_METRICS>
        <TASKOPERATIONS>
            <ProteinInterfaceDesign name="pack_long" design_chain1="0" design_chain2="0" jump="1" interface_distance_cutoff="15"/>
        </TASKOPERATIONS>
        <MOVERS>
            <TaskAwareMinMover name="min" scorefxn="sfxn" bb="0" chi="1" task_operations="pack_long" />
        </MOVERS>
        <FILTERS>
            <ContactMolecularSurface name="contact_patch" distance_weight="0.5" target_selector="patchdock_res" binder_selector="chainA" confidence="0" />
            <Ddg name="ddg"  threshold="-10" jump="1" repeats="5" repack="1" relax_mover="min" confidence="0" scorefxn="sfxn" extreme_value_removal="1" />
            <ResidueCount name="res_count_all" max_residue_count="9999" confidence="0"/>
            <BuriedSurfaceArea name="buried_npsa_FAMILYVW" select_only_FAMILYVW="True" atom_mode="hydrophobic_atoms" confidence="0.0" />
            <BuriedSurfaceArea name="buried_npsa" select_only_FAMILYVW="False" atom_mode="hydrophobic_atoms" confidence="0.0" />
            <ExposedHydrophobics name="exposed_hydrophobics"  confidence="0" />
            <CalculatorFilter name="buried_npsa_per_res" equation="total_score / res" threshold="-3.2" confidence="0">
                <Var name="total_score" filter="buried_npsa"/>
                <Var name="res" filter="res_count_all"/>
            </CalculatorFilter>
            <CalculatorFilter name="buried_npsa_FAMILYVW_per_res" equation="total_score / res" threshold="-3.2" confidence="0">
                <Var name="total_score" filter="buried_npsa_FAMILYVW"/>
                <Var name="res" filter="res_count_all"/>
            </CalculatorFilter>
            <MoveBeforeFilter name="buried_npsa_FAMILYVW_monomer" mover="chain1only" filter="buried_npsa_FAMILYVW" confidence="0" />
            <MoveBeforeFilter name="buried_npsa_monomer" mover="chain1only" filter="buried_npsa" confidence="0" />
            <MoveBeforeFilter name="exposed_hydrophobics_monomer" mover="chain1only" filter="exposed_hydrophobics" confidence="0" />
            <MoveBeforeFilter name="buried_npsa_per_res_monomer" mover="chain1only" filter="buried_npsa_per_res" confidence="0" />
            <MoveBeforeFilter name="buried_npsa_FAMILYVW_per_res_monomer" mover="chain1only" filter="buried_npsa_FAMILYVW_per_res" confidence="0" />
            <MoveBeforeFilter name="buried_npsa_FAMILYVW_apo" mover="move_chainA_far_away" filter="buried_npsa_FAMILYVW" confidence="0" />
            <MoveBeforeFilter name="buried_npsa_apo" mover="move_chainA_far_away" filter="buried_npsa" confidence="0" />
            <MoveBeforeFilter name="exposed_hydrophobics_apo" mover="move_chainA_far_away" filter="exposed_hydrophobics" confidence="0" />
            <CalculatorFilter   name="delta_buried_npsa_FAMILYVW" equation="complex - apo" threshold="-3.2" confidence="0">
                <Var name="complex" filter="buried_npsa_FAMILYVW"/>
                <Var name="apo"     filter="buried_npsa_FAMILYVW_apo"/>
            </CalculatorFilter>
            <CalculatorFilter   name="delta_buried_npsa" equation="complex - apo" threshold="-3.2" confidence="0">
                <Var name="complex" filter="buried_npsa"/>
                <Var name="apo"     filter="buried_npsa_apo"/>
            </CalculatorFilter>
            <CalculatorFilter   name="delta_exposed_hydrophobics" equation="complex - apo" threshold="-3.2" confidence="0">
                <Var name="complex" filter="exposed_hydrophobics"/>
                <Var name="apo"     filter="exposed_hydrophobics_apo"/>
            </CalculatorFilter>
            <ContactMolecularSurface name="contact_molecular_surface_ap_target" distance_weight="0.5" target_selector="apolar_B" binder_selector="chainA" confidence="0" />
            <ContactMolecularSurface name="contact_molec_sq5_ap_target" distance_weight="0.5" target_selector="apolar_B" binder_selector="chainA" confidence="0" near_squared_size="5" />
            <ContactMolecularSurface name="contact_molecular_surface_apap_target" distance_weight="0.5" target_selector="apolar_B" binder_selector="chainA" confidence="0" apolar_target="true" />
            <ContactMolecularSurface name="contact_molec_sq5_apap_target" distance_weight="0.5" target_selector="apolar_B" binder_selector="chainA" confidence="0" near_squared_size="5" apolar_target="true" />
            <ContactMolecularSurface name="contact_molecular_surface_ap_binder" distance_weight="0.5" target_selector="apolar_A" binder_selector="chainB" confidence="0" />
            <ContactMolecularSurface name="contact_molec_sq5_ap_binder" distance_weight="0.5" target_selector="apolar_A" binder_selector="chainB" confidence="0" near_squared_size="5" />
            <ContactMolecularSurface name="contact_molecular_surface_apap_binder" distance_weight="0.5" target_selector="apolar_A" binder_selector="chainB" confidence="0" apolar_target="true" />
            <ContactMolecularSurface name="contact_molec_sq5_apap_binder" distance_weight="0.5" target_selector="apolar_A" binder_selector="chainB" confidence="0" near_squared_size="5" apolar_target="true" />
            <worst9mer name="pre_wnm_all" rmsd_lookup_threshold="0.4" confidence="0" />
            <worst9mer name="pre_wnm_hlx" rmsd_lookup_threshold="0.4" confidence="0" only_helices="true" />
            <MoveBeforeFilter name="wnm_all" mover="chain1only" filter="pre_wnm_all" confidence="0" />
            <MoveBeforeFilter name="wnm_hlx" mover="chain1only" filter="pre_wnm_hlx" confidence="0" />
        </FILTERS>
        <PROTOCOLS>
            <Add filter="contact_molecular_surface_ap_target" />
            <Add filter="contact_molec_sq5_ap_target" />
            <Add filter="contact_molecular_surface_apap_target" />
            <Add filter="contact_molec_sq5_apap_target" />
            <Add filter="contact_patch"/>
            <Add filter="ddg"/>
            <Add filter="wnm_all"/>
            <Add filter="wnm_hlx"/>
            <Add metrics="sap_score" />
            <Add metrics="sap_score_target" />
            <Add metrics="binder_blocked_sap" />
            <Add metrics="target_blocked_sap" />
        </PROTOCOLS>
        <OUTPUT />
    </ROSETTASCRIPTS>
    """.format(
        interface_hydrophobics=scores["interface_hydrophobics"],
    )
    scorer = SingleoutputRosettaScriptsTask(score_xml)
    scored_ppose = scorer(designed_ppose.pose.clone())
    final_scores = dict(scored_ppose.pose.scores)
    scores.update(final_scores)
    scores["interface_hydrophobics"] = new_interface_hydrophobics
    scored_pose = io.to_pose(scored_ppose)
    for key, value in scores.items():
        pyrosetta.rosetta.core.pose.setPoseExtraScore(scored_pose, key, str(value))
    final_ppose = io.to_packed(scored_pose)
    return final_ppose

### We're going to do a huge run here, more than it makes sense to use dask for
Will submit everything to `backfill`
Make `SLURM` array tasks

In [ ]:
import os, stat, subprocess


def create_tasks(selected):
    with open(selected, "r") as f:
        for file in f:
            tasks = {}
            path = file.rstrip()
            tasks["-s"] = path
            yield tasks


def file_len(file):
    """https://stackoverflow.com/questions/845058/how-to-get-line-count-of-a-large-file-cheaply-in-python"""
    with open(file) as f:
        for i, l in enumerate(f):
            pass
    return i + 1


targets = os.path.join(os.getcwd(), "02_filter/to_detail.list")
detail = os.path.join(os.getcwd(), "detail.py")

jid = "{SLURM_JOB_ID%;*}"
sid = "{SLURM_ARRAY_TASK_ID}p"

for i, tasks in enumerate(tqdm(create_tasks(targets))):
    j = int(i / 99999)
    k = int(i / 1000)
    tasklist = f"03_detail_{j}.cmds"
    run_sh = """#!/usr/bin/env bash \n#SBATCH -J detail \n#SBATCH -e /mnt/home/pleung/logs/slurm_logs/detail-%J.err \n#SBATCH -o /mnt/home/pleung/logs/slurm_logs/detail-%J.out \n#SBATCH -p {queue} \n#SBATCH --mem=4G \n\nJOB_ID=${jid} \nCMD=$(sed -n "${sid}" {tasklist}) \necho "${c}" | bash""".format(
        queue="backfill", jid=jid, sid=sid, tasklist=tasklist, c="{CMD}"
    )
    shell = f"03_detail_{j}.sh"
    with open(shell, "w+") as f:
        print(run_sh, file=f)
    st = os.stat(shell)
    os.chmod(shell, st.st_mode | stat.S_IEXEC)
    with open(f"03_detail_{j}.cmds", "a+") as f:
        outpath = os.path.join(os.getcwd(), f"03_detail_{j}")
        full_outpath = os.path.join(os.getcwd(), outpath, f"{k}".zfill(4))
        args_ = " ".join([" ".join([k, str(v)]) for k, v in tasks.items()])
        cmd = f"mkdir -p {full_outpath}; cd {full_outpath}; {detail} {args_}"
        print(cmd, file=f)

# Let's go
print("Run the following commands")
for i in range(5):
    print(f"sbatch -a 1-$(cat 03_detail_{i}.cmds | wc -l) 03_detail_{i}.sh")

370598it [04:11, 1503.60it/s]

### Plot metrics by target

In [3]:
def row2state(row):
    state = (
        row["parent"]
        + "_p_"
        + str(int(row["pivot_helix"]))
        + "_s_"
        + str(int(row["shift"]))
        + "_d_"
        + str(int(row["docked_helix"]))
    )
    return state


for _, df in target_dfs.items():
    df["state"] = df.apply(row2state, axis=1)

In [4]:
sns.set(
    context="talk",
    font_scale=1.5,  # make the font larger; default is pretty small
    style="ticks",  # make the background white with black lines
    palette="colorblind",  # a color palette that is colorblind friendly!
)


def rho(x, y, ax=None, **kwargs):
    """Plot the correlation coefficient in the top left hand corner of a plot.
    https://stackoverflow.com/questions/50832204/show-correlation-values-in-pairplot-using-seaborn-in-python/50835066
    """
    import scipy

    r, _ = scipy.stats.pearsonr(x, y)
    ax = ax or plt.gca()
    # Unicode for lowercase rho (ρ)
    rho = "\u03C1"
    ax.annotate(f"{rho} = {r:.2f}", xy=(0.1, 0.9), xycoords=ax.transAxes)


def plot_unity(xdata, ydata, **kwargs):
    """https://stackoverflow.com/questions/48122019/how-can-i-plot-identity-lines-on-a-seaborn-pairplot"""
    xmin, ymin = (xdata.min(), ydata.min())
    xmax, ymax = (xdata.max(), ydata.max())
    xpoints = np.linspace(xmin, xmax, 100)
    ypoints = np.linspace(ymin, ymax, 100)
    plt.gca().plot(
        xpoints, ypoints, color="k", marker=None, linestyle="--", linewidth=4.0
    )


for target, df in tqdm(target_dfs.items()):
    subset = df[df["ddg"] < 0]
    subset = subset[subset["ddg_hydrophobic"] < 0]

    subset = subset[
        [
            "contact_molecular_surface",
            "contact_patch",
            "ddg",
            "ddg_hydrophobic",
            "dslf_fa13",
            "fa_atr_pocket",
            "hydrophobic_residue_contacts",
            "interface_buried_sasa",
            "interface_sc",
            "score_per_res",
            "ss_sc",
            "vbuns5.0_heavy_ball_1.1D",
            "parent",
            "state",
            "target_name",
        ]
    ]

    pack_subset = subset[
        [
            "contact_molecular_surface",
            "contact_patch",
            "ddg",
            "ddg_hydrophobic",
            "fa_atr_pocket",
            "hydrophobic_residue_contacts",
            "interface_buried_sasa",
            "interface_sc",
            "parent",
            "state",
            "target_name",
        ]
    ]

    ax = sns.pairplot(
        data=pack_subset.sample(frac=0.01), hue="parent", corner=True, height=8
    )
    plt.suptitle(f"{target}\nCorrelation of interface metrics, split by parent")
    sns.despine()
    plt.savefig(f"figs/01_{target}_correlations_int_split_by_parent.png")
    plt.close()
    ax = sns.pairplot(data=pack_subset.sample(frac=0.01), corner=True, height=8)
    ax.map_lower(rho)
    ax.map_offdiag(plot_unity)
    plt.suptitle(f"{target}\nCorrelation of fragment metrics, with pearson R")
    sns.despine()
    plt.savefig(f"figs/01_{target}_correlations_int_pearson.png")
    plt.close()

  0%|          | 0/7 [00:00<?, ?it/s]/home/pleung/.conda/envs/phil/lib/python3.8/site-packages/seaborn/distributions.py:306: UserWarning: Dataset has 0 variance; skipping density estimate.
  warnings.warn(msg, UserWarning)
/home/pleung/.conda/envs/phil/lib/python3.8/site-packages/seaborn/distributions.py:306: UserWarning: Dataset has 0 variance; skipping density estimate.
  warnings.warn(msg, UserWarning)
/home/pleung/.conda/envs/phil/lib/python3.8/site-packages/seaborn/distributions.py:306: UserWarning: Dataset has 0 variance; skipping density estimate.
  warnings.warn(msg, UserWarning)
/home/pleung/.conda/envs/phil/lib/python3.8/site-packages/seaborn/distributions.py:306: UserWarning: Dataset has 0 variance; skipping density estimate.
  warnings.warn(msg, UserWarning)
/home/pleung/.conda/envs/phil/lib/python3.8/site-packages/seaborn/distributions.py:306: UserWarning: Dataset has 0 variance; skipping density estimate.
  warnings.warn(msg, UserWarning)
/home/pleung/.conda/envs/phil/lib

### Get designs that pass reasonable filters by target and save pickle of dict of filtered dfs

In [5]:
filtered_dfs = {}
output_path = os.path.join(os.getcwd(), "01_filter")
for target, df in target_dfs.items():
    filtered = df[df["ddg"] < -40]
    filtered = filtered[filtered["interface_buried_sasa"] > 1400]
    filtered = filtered[filtered["score_per_res"] < -3]
    filtered = filtered[filtered["wnm_all"] < 0.8]
    filtered = filtered[filtered["wnm_hlx"] < 0.15]
    print(target)
    print(len(set(filtered.state.values)))
    print(len(filtered))
    print(len(set(filtered.parent.values)))
    print(set(df.parent.values) - set(filtered.parent.values))
    filtered_dfs[target] = filtered

with open(os.path.join(os.getcwd(), "01_filter", "filtered_dfs.pkl"), "wb") as handle:
    pickle.dump(filtered_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)

apoe
160
21787
46
{'KH_PXX28_AWP', 'hDHR53_5CWK', 'THR13', 'hDHR54_5CWL', 'THR7', 'hDHR14_5H7C', 'THR9', 'DHR39', 'THR4', 'THR11', 'DHR79', 'DHR53', 'hTHR_37'}
covstem
93
2122
37
{'THR7', 'DHR68', 'THR4', 'hDHR71_5CWN', 'THR3H_4', 'KH_R4_PXX13', 'hDHR54_5CWL', 'TH_DHR_2_NSR', 'DHR79', 'DHR53', 'TH_DHR_C9', 'DHR52_nocys', 'hDHR14_5H7C', 'KH_PXX28_AWP', 'hDHR53_5CWK', 'THR13', 'hDHR49_5CWJ', 'KH_PEP12_PLP', 'DHR57', 'THR11', 'THR9', 'hTHR_37'}
gip
180
16681
53
{'hDHR53_5CWK', 'THR9', 'THR4', 'DHR79', 'DHR53', 'hTHR_37'}
glp
179
41186
49
{'hDHR53_5CWK', 'THR9', 'THR4', 'THR11', 'hDHR71_5CWN', 'THR3H_4', 'DHR79', 'hTHR_37'}
glucagon
174
19068
48
{'hDHR53_5CWK', 'THR9', 'THR4', 'THR11', 'hDHR71_5CWN', 'THR3H_4', 'DHR79', 'DHR53', 'hTHR_37'}
neuropeptideY
191
17287
51
{'hDHR53_5CWK', 'THR9', 'KH_PEP12_PLP', 'THR4', 'THR11', 'DHR79', 'DHR53', 'hTHR_37'}
pth
180
20759
51
{'hDHR53_5CWK', 'THR9', 'THR4', 'THR11', 'DHR79', 'DHR53'}


### Save a list of the input docks/threads that resulted in designs that passed filters

In [7]:
inputs_to_rerun = []
for target, df in filtered_dfs.items():
    for input_path in set(df.file_in.values):
        inputs_to_rerun.append(input_path)
inputs_to_rerun = set(inputs_to_rerun)
with open(os.path.join(os.getcwd(), "01_filter", "rerun.list"), "w+") as f:
    for decoy in inputs_to_rerun:
        print(decoy, file=f)

### Save a list of the designs that passed filters in the first round. This will be combined with designs that pass after reruns

In [8]:
passed_filters = []
for target, df in filtered_dfs.items():
    for decoy in df.index:
        passed_filters.append(decoy)
with open(os.path.join(os.getcwd(), "01_filter", "passed.list"), "w+") as f:
    for decoy in passed_filters:
        print(decoy, file=f)

In [9]:
!mv 01*.cmds 01_filter; mv 01*.sh 01_filter

### Unused blocks

In [ ]:
%%time
import pyrosetta
from pyrosetta.distributed import cluster
import pyrosetta.distributed.io as io

flags = """
-out:level 300
-corrections::beta_nov16 true
-holes:dalphaball /home/bcov/ppi/tutorial_build/main/source/external/DAlpahBall/DAlphaBall.gcc
-indexed_structure_store:fragment_store /home/bcov/sc/scaffold_comparison/data/ss_grouped_vall_all.h5
"""
# pyrosetta.distributed.init(" ".join(flags.replace("\n\t", " ").split()))
pyrosetta.init(" ".join(flags.replace("\n\t", " ").split()))

t = detail_design(
    None,
    **{
        "-s": "/mnt/home/pleung/projects/peptide_binders/r0/peptide_binders/02_filter/decoys/0/0cdba3156384aca724430067c0922d72a63ec833ba196a52.pdb.bz2",
    }
)